In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely
from shapely.wkt import loads

### 天然气管道

In [2]:
df1=pd.read_excel("../data/existing_pipelines/GEM-GGIT-Gas-Pipelines-December-2022.xlsx")

In [3]:
df2=df1.loc[(df1["StartCountry"]=="China")&(df1["EndCountry"]=="China")]

In [4]:
df2=df2[["PipelineName","SegmentName","Status","StartLocation","EndLocation","Capacity","CapacityUnits","StartState/Province","EndState/Province","WKTFormat"]]
df2

,PipelineName,SegmentName,Status,StartLocation,EndLocation,Capacity,CapacityUnits,StartState/Province,EndState/Province,WKTFormat
479,Yacheng-Hong Kong Gas Pipeline,NaN,Operating,Yacheng,NaN,3.4,bcm/y,Hainan,Hong Kong,"LINESTRING (109.117774 17.289356, 114.118131 2..."
2529,Ningxia Gas Pipeline Network,Hangjinqi-Yinchuan Gas Pipeline,Operating,Hanggin,NaN,2.5,bcm/y,Inner Mongolia,Ningxia,"LINESTRING (108.810472 39.61288, 107.270398 39..."
2530,Ningxia Gas Pipeline Network,Yinchuan-Shizuishan parallel gas pipeline,Proposed,NaN,NaN,1.6,bcm/y,Ningxia,Ningxia,"LINESTRING (106.110757 38.456894, 106.67360395..."
2531,Ningxia Gas Pipeline Network,Yinchuan-Wuzhong gas pipeline,Proposed,NaN,NaN,NaN,NaN,Ningxia,Ningxia,"LINESTRING (106.110757 38.456894, 106.18381622..."
2532,Ningxia Gas Pipeline Network,Yanchi-Yinchuan gas pipeline,Proposed,NaN,NaN,NaN,NaN,Ningxia,Ningxia,"LINESTRING (106.110757 38.456894, 107.35866637..."
...,...,...,...,...,...,...,...,...,...,...
3138,Hainan Haikou–Guangdong Xvwen Pipeline,NaN,Shelved,NaN,Xvwen,10,bcm/y,Hainan,Guangdong,"LINESTRING (110.362444 21.266637, 110.197253 2..."
3139,Shenmu-Anping Coalbed Gas Pipeline,Shaanxi-Shanxi Segment,Construction,Kangning,NaN,2,bcm/y,Shaanxi,Shanxi,"LINESTRING (111.078255 38.335616, 111.07578 38..."
3140,Shenmu-Anping Coalbed Gas Pipeline,Shanxi–Hebei Segment,Operating,Kangning,Anping,5,bcm/y,Shaanxi,Hebei,"LINESTRING (111.078255 38.335616, 111.097273 3..."
3141,Jiangsu–Henan–Anhui Provinces Connection Line ...,NaN,Proposed,Binhai,NaN,15,bcm/y,Jiangsu,Henan,"LINESTRING (119.997 34.096, 116.964 33.648, 11..."


查找单位不统一的，结果都是省内的，不用管

In [5]:
df2[(df2["CapacityUnits"]!="bcm/y")&(~df2["CapacityUnits"].isnull())] #billion cubic meters per year 十亿立方米/年
#MMSCMD,Million Standard Cubic Metre per Day,*365/1000即转换为bcm
#104cm/d,万立方米/天，*365/100000即转换为bcm



,PipelineName,SegmentName,Status,StartLocation,EndLocation,Capacity,CapacityUnits,StartState/Province,EndState/Province,WKTFormat
2722,Sichuan & Chongqing Grand Loop Line,Zhongjiang-Longquanyi Gas Pipeline,Operating,NaN,NaN,200,104cm/d,Sichuan,Sichuan,--
3058,Chongqing Gas Pipeline Network,Da County-Wolonghe pipeline 达卧线（达县柳垭-卧龙河）,Operating,NaN,NaN,2.5,MMSCMD,Chongqing,Chongqing,--
3059,Chongqing Gas Pipeline Network,Ba County-Chongqing gas pipeline,Operating,NaN,NaN,2.5,MMSCMD,Chongqing,Chongqing,--
3060,Chongqing Gas Pipeline Network,Wolonghe-Chongqing gas pipeline,Operating,NaN,NaN,2.5,MMSCMD,Chongqing,Chongqing,--


跨省的管道，补全起始、终点省份缺失

In [6]:
df_cross=df2[df2["StartState/Province"]!=df2["EndState/Province"]]
df_cross


,PipelineName,SegmentName,Status,StartLocation,EndLocation,Capacity,CapacityUnits,StartState/Province,EndState/Province,WKTFormat
479,Yacheng-Hong Kong Gas Pipeline,NaN,Operating,Yacheng,NaN,3.4,bcm/y,Hainan,Hong Kong,"LINESTRING (109.117774 17.289356, 114.118131 2..."
2529,Ningxia Gas Pipeline Network,Hangjinqi-Yinchuan Gas Pipeline,Operating,Hanggin,NaN,2.5,bcm/y,Inner Mongolia,Ningxia,"LINESTRING (108.810472 39.61288, 107.270398 39..."
2533,China–Russia East Pipeline,NaN,Operating,Changling,Yongqing,38,bcm/y,Jilin,Hebei,"LINESTRING (123.644593 43.35972, 123.485958 43..."
2534,China–Russia East Pipeline,Phase II,Operating,Changling,Yongqing,38,bcm/y,Jilin,Hebei,"LINESTRING (123.644593 43.35972, 123.485958 43..."
2535,China–Russia East Pipeline,Phase I,Operating,Heihe,Changling,38,bcm/y,Heilongjiang,Jilin,"LINESTRING (127.529918 50.24344, 127.36687 50...."
...,...,...,...,...,...,...,...,...,...,...
3138,Hainan Haikou–Guangdong Xvwen Pipeline,NaN,Shelved,NaN,Xvwen,10,bcm/y,Hainan,Guangdong,"LINESTRING (110.362444 21.266637, 110.197253 2..."
3139,Shenmu-Anping Coalbed Gas Pipeline,Shaanxi-Shanxi Segment,Construction,Kangning,NaN,2,bcm/y,Shaanxi,Shanxi,"LINESTRING (111.078255 38.335616, 111.07578 38..."
3140,Shenmu-Anping Coalbed Gas Pipeline,Shanxi–Hebei Segment,Operating,Kangning,Anping,5,bcm/y,Shaanxi,Hebei,"LINESTRING (111.078255 38.335616, 111.097273 3..."
3141,Jiangsu–Henan–Anhui Provinces Connection Line ...,NaN,Proposed,Binhai,NaN,15,bcm/y,Jiangsu,Henan,"LINESTRING (119.997 34.096, 116.964 33.648, 11..."


In [14]:
df_cross_1=df_cross[~((df_cross["WKTFormat"]=='--')|(df_cross["Capacity"].isnull()))] #把容量和坐标都没有的去掉
df_cross_1.shape

(80, 10)

In [15]:
df_miss=df_cross_1[df_cross_1['StartState/Province'].isna()|df_cross_1['EndState/Province'].isna()] #剩下没有坐标的，看看是否是省内


,PipelineName,SegmentName,Status,StartLocation,EndLocation,Capacity,CapacityUnits,StartState/Province,EndState/Province,WKTFormat
2603,Inner Mongolia Gas Pipeline Network,Sumitu-Dongsheng-Junger Gas Pipeline,Operating,NaN,Junggar,0.47,bcm/y,NaN,Inner Mongolia,"LINESTRING (108.393015 38.615227, 109.973494 3..."
3127,Pinghu-Shanghai Offshore Gas Pipeline,NaN,Operating,East Sea,Xingang Town,1.09,bcm/y,NaN,Shanghai,"LINESTRING (121.871103 30.996338, 126.309621 2..."


In [16]:
 #都是省内的,删掉
df_cross_1.dropna(subset=['StartState/Province','EndState/Province'],inplace=True)
df_cross_1.shape

C:\Users\ASUS\AppData\Local\Temp\ipykernel_28916\3787415989.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cross_1.dropna(subset=['StartState/Province','EndState/Province'],inplace=True)


(78, 10)

#### 补全容量，统一容量单位

In [17]:
df_cross_1["CapacityUnits"]=df_cross_1["CapacityUnits"].fillna("bcm/y")

C:\Users\ASUS\AppData\Local\Temp\ipykernel_28916\927471109.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cross_1["CapacityUnits"]=df_cross_1["CapacityUnits"].fillna("bcm/y")


In [31]:
Capacity_h2=1e10*0.1*5.28
Capacity_h2

5280000000.0

In [18]:
#typro error
df_cross_1["StartState/Province"]= df_cross_1["StartState/Province"].replace('Heibei','Hebei').replace('Guangzhou','Guangdong').replace("jiangxi","Jiangxi")

C:\Users\ASUS\AppData\Local\Temp\ipykernel_28916\2844763502.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cross_1["StartState/Province"]= df_cross_1["StartState/Province"].replace('Heibei','Hebei').replace('Guangzhou','Guangdong').replace("jiangxi","Jiangxi")


异常处理,有一个终点有三个省份的，直接删掉

In [19]:
df_cross_1[df_cross_1["EndState/Province"]=='Heibei, Henan, Tianjin']


,PipelineName,SegmentName,Status,StartLocation,EndLocation,Capacity,CapacityUnits,StartState/Province,EndState/Province,WKTFormat
3055,Ordos-Anping-Cangzhou Gas Pipeline,NaN,Operating,"尔林兔 Town, Shenmu County","Cang County, 旧州镇, Qingyuan, 文留镇, Puyang, Dagang",30,bcm/y,Shaanxi,"Heibei, Henan, Tianjin","MULTILINESTRING ((109.947011 39.051613, 111.29..."


In [20]:
df_cross_1.drop(index=3055,inplace=True)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_28916\492807500.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cross_1.drop(index=3055,inplace=True)


In [25]:
df_cross_1.sort_values(by="Capacity",ascending=False)

,PipelineName,SegmentName,Status,StartLocation,EndLocation,Capacity,CapacityUnits,StartState/Province,EndState/Province,WKTFormat,capacity_h2
2995,West-East Gas Pipeline 5,NaN,Proposed,Ulugqat,NaN,45,bcm/y,Xinjiang,Ningxia,"LINESTRING (73.9261698 39.69572899, 73.961375 ...",36307611152.423355
2533,China–Russia East Pipeline,NaN,Operating,Changling,Yongqing,38,bcm/y,Jilin,Hebei,"LINESTRING (123.644593 43.35972, 123.485958 43...",30659760528.713055
2534,China–Russia East Pipeline,Phase II,Operating,Changling,Yongqing,38,bcm/y,Jilin,Hebei,"LINESTRING (123.644593 43.35972, 123.485958 43...",30659760528.713055
2535,China–Russia East Pipeline,Phase I,Operating,Heihe,Changling,38,bcm/y,Heilongjiang,Jilin,"LINESTRING (127.529918 50.24344, 127.36687 50....",30659760528.713055
2537,China–Russia East Pipeline,Phase III,Construction,Yongqing,Baihe,38,bcm/y,Hebei,Shanghai,"MULTILINESTRING ((116.51979 39.297164, 115.429...",30659760528.713055
...,...,...,...,...,...,...,...,...,...,...,...
2952,Hunan Gas Pipeline Network,Huarong-Nanxian-Anxiang Gas Pipeline,Operating,Shishou,NaN,0.6,bcm/y,Hubei,Hunan,"LINESTRING (112.428092 29.69337, 112.48652 29....",484101482.032311
2611,Jilin Gas Pipeline Network,Songyuan–Baicheng–Ulanhot Gas Pipeline,Proposed,Qianguo County,NaN,0.58,bcm/y,Jilin,Inner Mongolia,"LINESTRING (124.786273 45.195823, 122.862874 4...",467964765.964568
3042,Liaoning Gas Pipeline Network,Jianping-Chifeng Gas Pipeline,Construction,Jianping,Hongshan District,0.53,bcm/y,Liaoning,Inner Mongolia,"LINESTRING (119.721011 41.398684, 119.334502 4...",427622975.795208
2854,Hebei Gas Pipeline Network,Ying County–Zhangjiakou Gas Pipeline,Operating,Ying County,Zhangjiakou,0.47,bcm/y,Shanxi,Hebei,"LINESTRING (113.203268 39.554038, 114.900897 4...",379212827.591977


天然气体积转换成可输氢质量

In [22]:
#根据我国天然气的输气压力，可将天然气管道分为高压管道、中压管道及低压管道。高压管道一般为全国战略性管道，如西气东输管道、中亚天然气管道等，其压力一般为10～12MPa;中压管道一般为区内干线、联络线和干线配套支线管道等，设计压力为4～12MPa;低压管道一般为城市燃气管道，其压力低于4MPa。根据现有的我国天然气掺氢项目，http://www.chinaden.cn/news_nr.asp?id=31208&Small_Class=3，将压力暂定为5MPa

pressure= 5e6 #Pa,压缩系数1.0297


In [23]:
density_hydrogen=3.89 #kg/m3

4.034179016935928

In [24]:
fraction=0.2
df_cross_1['capacity_h2']=density_hydrogen*fraction*df_cross_1["Capacity"]*1e9 #单位转换为kg/y
df_cross_1

C:\Users\ASUS\AppData\Local\Temp\ipykernel_28916\3140847121.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cross_1['capacity_h2']=density_hydrogen*fraction*df_cross_1["Capacity"]*1e9 #单位转换为kg/y


,PipelineName,SegmentName,Status,StartLocation,EndLocation,Capacity,CapacityUnits,StartState/Province,EndState/Province,WKTFormat,capacity_h2
479,Yacheng-Hong Kong Gas Pipeline,NaN,Operating,Yacheng,NaN,3.4,bcm/y,Hainan,Hong Kong,"LINESTRING (109.117774 17.289356, 114.118131 2...",2743241731.516431
2529,Ningxia Gas Pipeline Network,Hangjinqi-Yinchuan Gas Pipeline,Operating,Hanggin,NaN,2.5,bcm/y,Inner Mongolia,Ningxia,"LINESTRING (108.810472 39.61288, 107.270398 39...",2017089508.467964
2533,China–Russia East Pipeline,NaN,Operating,Changling,Yongqing,38,bcm/y,Jilin,Hebei,"LINESTRING (123.644593 43.35972, 123.485958 43...",30659760528.713055
2534,China–Russia East Pipeline,Phase II,Operating,Changling,Yongqing,38,bcm/y,Jilin,Hebei,"LINESTRING (123.644593 43.35972, 123.485958 43...",30659760528.713055
2535,China–Russia East Pipeline,Phase I,Operating,Heihe,Changling,38,bcm/y,Heilongjiang,Jilin,"LINESTRING (127.529918 50.24344, 127.36687 50....",30659760528.713055
...,...,...,...,...,...,...,...,...,...,...,...
3138,Hainan Haikou–Guangdong Xvwen Pipeline,NaN,Shelved,NaN,Xvwen,10,bcm/y,Hainan,Guangdong,"LINESTRING (110.362444 21.266637, 110.197253 2...",8068358033.871857
3139,Shenmu-Anping Coalbed Gas Pipeline,Shaanxi-Shanxi Segment,Construction,Kangning,NaN,2,bcm/y,Shaanxi,Shanxi,"LINESTRING (111.078255 38.335616, 111.07578 38...",1613671606.774371
3140,Shenmu-Anping Coalbed Gas Pipeline,Shanxi–Hebei Segment,Operating,Kangning,Anping,5,bcm/y,Shaanxi,Hebei,"LINESTRING (111.078255 38.335616, 111.097273 3...",4034179016.935928
3141,Jiangsu–Henan–Anhui Provinces Connection Line ...,NaN,Proposed,Binhai,NaN,15,bcm/y,Jiangsu,Henan,"LINESTRING (119.997 34.096, 116.964 33.648, 11...",12102537050.807785


天然气管道整理

In [26]:
from src.const import list_provincial_level

In [27]:
NGtable= pd.DataFrame(
  columns=list_provincial_level, # 指定列属性
  index=list_provincial_level  # 指定行索引
) 
NGtable.fillna(0,inplace=True)

In [28]:
df_cross_1.shape

(77, 11)

In [29]:
for i in range(len(df_cross_1)):
    index=df_cross_1.iloc[i]["StartState/Province"]	#index是起点省份，column是终点省份
    column=df_cross_1.iloc[i]["EndState/Province"]
    NGtable.loc[index,column]=NGtable.loc[index,column]+df_cross_1.iloc[i]["capacity_h2"] #累加容量

In [30]:
NGtable

,Inner Mongolia,Heilongjiang,Jilin,Liaoning,Gansu,Ningxia,Qinghai,Shaanxi,Xinjiang,Beijing,...,Hunan,Jiangxi,Anhui,Fujian,Jiangsu,Shanghai,Zhejiang,Taiwan,Hong Kong,Macao
Inner Mongolia,0.000000e+00,0,0.000000e+00,0.000000e+00,0.000000e+00,2.331755e+09,0,0.000000e+00,0,6.761284e+09,...,0.000000e+00,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0.000000e+00,0
Heilongjiang,6.051269e+08,0,3.065976e+10,0.000000e+00,0.000000e+00,0.000000e+00,0,0.000000e+00,0,0.000000e+00,...,0.000000e+00,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0.000000e+00,0
Jilin,4.679648e+08,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0.000000e+00,0,0.000000e+00,...,0.000000e+00,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0.000000e+00,0
Liaoning,1.879927e+09,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0.000000e+00,0,0.000000e+00,...,0.000000e+00,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0.000000e+00,0
Gansu,0.000000e+00,0,0.000000e+00,0.000000e+00,0.000000e+00,2.823925e+09,0,0.000000e+00,0,0.000000e+00,...,0.000000e+00,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0.000000e+00,0
Ningxia,0.000000e+00,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,2.420507e+10,0,0.000000e+00,...,0.000000e+00,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0.000000e+00,0
Qinghai,0.000000e+00,0,0.000000e+00,0.000000e+00,4.276230e+09,0.000000e+00,0,0.000000e+00,0,0.000000e+00,...,0.000000e+00,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0.000000e+00,0
Shaanxi,0.000000e+00,0,0.000000e+00,0.000000e+00,0.000000e+00,8.713827e+08,0,0.000000e+00,0,4.518280e+10,...,0.000000e+00,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0.000000e+00,0
Xinjiang,0.000000e+00,0,0.000000e+00,0.000000e+00,0.000000e+00,4.841015e+10,0,0.000000e+00,0,0.000000e+00,...,0.000000e+00,0,0.000000e+00,6.858104e+10,0.000000e+00,1.371621e+10,0.000000e+00,0,0.000000e+00,0
Beijing,0.000000e+00,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0.000000e+00,0,0.000000e+00,...,0.000000e+00,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0.000000e+00,0


In [31]:
NGtable.to_csv("../data/existing_pipelines/NGtable.csv")

将几何形状存在的df_cross_lines变为gpd,用于qgis可视化

In [32]:
# from shapely.geometry import LineString
# def create_geometry(row):
#     if row != '--':
#         return row
#     else:
#         # 如果数据为空，返回一个空的 LineString
#         return 'LINESTRING EMPTY'

# 应用函数创建新的几何列
df_cross_1.rename(columns={'WKTFormat': 'geometry'}, inplace=True)
gdf_cross_new=df_cross_1[['PipelineName','StartState/Province','EndState/Province','Status','capacity_h2','geometry']].copy()

gdf_cross_new=gpd.GeoDataFrame(gdf_cross_new,geometry=gdf_cross_new["geometry"].apply(loads),crs="EPSG:4326")
gdf_cross_new.reset_index(drop=True,inplace=True)
gdf_cross_new.rename(columns={'StartState/Province':'start','EndState/Province':'end'},inplace=True)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_28916\1399735592.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cross_1.rename(columns={'WKTFormat': 'geometry'}, inplace=True)


In [34]:
import math
def haversine(s, s_n):
    # 将经纬度转换成弧度
    list_s=list(s)
    list_s_n=list(s_n)
    list_s[1], list_s[0], list_s_n[1], list_s_n[0] = map(math.radians, [s[1], s[0], s_n[1], s_n[0]])

    # 计算纬度和经度的差值
    dlat = list_s_n[1] - list_s[1]
    dlon = list_s_n[0] - list_s[0]

    # 应用Haversine公式
    a = math.sin(dlat / 2)**2 + math.cos(list_s[1]) * math.cos(list_s_n[1]) * math.sin(dlon / 2)**2
    c = 2 * math.asin(math.sqrt(a))

    # 地球的平均半径（公里）
    r = 6371

    # 计算结果,单位为公里
    return c*r

In [35]:
from shapely.geometry import MultiLineString, LineString

def calculate_length(geometry):
    if isinstance(geometry, LineString):
        if geometry.is_empty:
            # 如果是空的，返回 0
            return 0
        # 如果是 LineString，计算长度
        else:
            return sum(haversine(geometry.coords[i], geometry.coords[i+1]) for i in range(len(geometry.coords)-1))
    elif isinstance(geometry, MultiLineString):
        # 如果是 MultiLineString，迭代每个 LineString 并计算长度
        return sum(calculate_length(line) for line in geometry.geoms)
    else:
        # 如果不是 LineString 或 MultiLineString，返回 0 或适当的值
        return 0

# 应用函数计算每个几何对象的长度
gdf_cross_new['length'] = gdf_cross_new.geometry.apply(calculate_length)


In [36]:
gdf_cross_new[gdf_cross_new['length']==0]

,PipelineName,start,end,Status,capacity_h2,geometry,length


In [39]:
gdf_cross_new['capacity_h2']=gdf_cross_new['capacity_h2']/1e7

In [46]:
gdf_cross_new['capacity_h2'] = pd.to_numeric(gdf_cross_new['capacity_h2'], errors='coerce')

In [49]:
gdf_cross_new

,PipelineName,start,end,Status,capacity_h2,geometry,length
0,Yacheng-Hong Kong Gas Pipeline,Hainan,Hong Kong,Operating,274.324173,"LINESTRING (109.11777 17.28936, 114.11813 22.4...",779.056046
1,Ningxia Gas Pipeline Network,Inner Mongolia,Ningxia,Operating,201.708951,"LINESTRING (108.81047 39.61288, 107.27040 39.2...",275.676769
2,China–Russia East Pipeline,Jilin,Hebei,Operating,3065.976053,"LINESTRING (123.64459 43.35972, 123.48596 43.0...",805.114571
3,China–Russia East Pipeline,Jilin,Hebei,Operating,3065.976053,"LINESTRING (123.64459 43.35972, 123.48596 43.0...",805.114571
4,China–Russia East Pipeline,Heilongjiang,Jilin,Operating,3065.976053,"LINESTRING (127.52992 50.24344, 127.36687 50.1...",834.655514
...,...,...,...,...,...,...,...
72,Hainan Haikou–Guangdong Xvwen Pipeline,Hainan,Guangdong,Shelved,806.835803,"LINESTRING (110.36244 21.26664, 110.19725 20.0...",137.280495
73,Shenmu-Anping Coalbed Gas Pipeline,Shaanxi,Shanxi,Construction,161.367161,"LINESTRING (111.07825 38.33562, 111.07578 38.3...",60.352701
74,Shenmu-Anping Coalbed Gas Pipeline,Shaanxi,Hebei,Operating,403.417902,"LINESTRING (111.07825 38.33562, 111.09727 38.3...",491.346167
75,Jiangsu–Henan–Anhui Provinces Connection Line ...,Jiangsu,Henan,Proposed,1210.253705,"LINESTRING (119.99700 34.09600, 116.96400 33.6...",432.054490


In [48]:

gdf_cross_new.to_file('../data/existing_pipelines/gpf_cross_new.geojson',driver='GeoJSON')


#### 将省i到省j的纯氢管道容量整理成表

In [94]:
PHtable=pd.DataFrame(
  columns=list_provincial_level, # 指定列属性
  index=list_provincial_level  # 指定行索引
) 
PHtable.fillna(0,inplace=True)

In [95]:
PHtable.loc["Inner Mongolia","Beijing"]=500000000  #单位：kg/y

In [96]:
PHtable

,Inner Mongolia,Heilongjiang,Jilin,Liaoning,Gansu,Ningxia,Qinghai,Shaanxi,Xinjiang,Beijing,...,Hunan,Jiangxi,Anhui,Fujian,Jiangsu,Shanghai,Zhejiang,Taiwan,Hong Kong,Macao
Inner Mongolia,0,0,0,0,0,0,0,0,0,500000000,...,0,0,0,0,0,0,0,0,0,0
Heilongjiang,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Jilin,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Liaoning,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Gansu,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ningxia,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Qinghai,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Shaanxi,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Xinjiang,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Beijing,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [97]:
PHtable.to_csv("../data/existing_pipelines/PHtable.csv")